In [46]:
import typing
import json

import requests

import tqdm
import pandas as pd

import config

In [47]:
CFG = config.Config()

In [48]:
MODEL: str = "mixtral:8x7b-instruct-v0.1-q6_K"

TASK_NAME: str = "political_ideology"
TASK_TEMPLATE: str = \
"""
Classify the following message as ideologically liberal (0), ideologically neutral (1), or ideologically conservative (2). Ideology here is defined in the context of the German political system. Messages with no ideological content are classified as neutral. 

Respond with only the predicted class (0 or 1 or 2) of the request.

Text: {text}
Class:
"""

In [49]:
user_content: pd.DataFrame = pd.read_parquet(CFG.final_files["user_content"])
user_content

content  \
userId                   id                                                                            
661d1639b8beabb58229451b 66252255616a4cb7061ee32c  #UkraineKrieg: Beunruhigende Entwicklungen in ...   
                         66253e74616a4cb7061ee7cc  Bedauerlich, dass gewaltsame Konflikte weiterh...   
                         66255134616a4cb7061eeaf4  Gewalt im Nahen Osten & Ukraine nicht isoliert...   
                         6627d8efa6b56b19b4300ae4  Unterstützen wir unsere Verbündeten, respektie...   
                         6628b08fa6b56b19b4301867  Geschlossene Tür für Unsicherheit: US-Repräsen...   
...                                                                                              ...   
662a070537d6395f42ca7bf9 662a070537d6395f42ca7bfd       Kleiner vermieter Junge noch nicht gefunden    
662a12fa37d6395f42ca87fa 662a12fa37d6395f42ca87fe         Lok Leipzig holt Jochen Seitz als Trainer    
662a30f137d6395f42ca99c2 662a30f137d6395f42ca99c6                             Russland droht der USA   
662e892b3ae8346ce92c8e81 662e892b3ae8346ce92c8e85                               Arian wird vermisst    
662f64073ae8346ce92c9d21 662f64083ae8346ce92c9d25                                            Zinsen    

                                                                 createdAt  
userId                   id                                                 
661d1639b8beabb58229451b 66252255616a4cb7061ee32c  2024-04-21 14:27:33.315  
                         66253e74616a4cb7061ee7cc  2024-04-21 16:27:32.919  
                         66255134616a4cb7061eeaf4  2024-04-21 17:47:32.762  
                         6627d8efa6b56b19b4300ae4  2024-04-23 15:51:11.067  
                         6628b08fa6b56b19b4301867  2024-04-24 07:11:11.797  
...                                                                    ...  
662a070537d6395f42ca7bf9 662a070537d6395f42ca7bfd  2024-04-25 07:32:21.584  
662a12fa37d6395f42ca87fa 662a12fa37d6395f42ca87fe  2024-04-25 08:23:22.984  
662a30f137d6395f42ca99c2 662a30f137d6395f42ca99c6  2024-04-25 10:31:13.258  
662e892b3ae8346ce92c8e81 662e892b3ae8346ce92c8e85  2024-04-28 17:36:43.427  
662f64073ae8346ce92c9d21 662f64083ae8346ce92c9d25  2024-04-29 09:10:32.138  

[592 rows x 2 columns]

In [50]:
predictions: typing.Dict[str, str] = {}

In [ ]:
for index, row in tqdm.tqdm(user_content.iterrows()):
    try: 
        predictions[index] = requests.post(
        'https://inf.cl.uni-trier.de/',
        json={
            'model': MODEL,
            'system': TASK_TEMPLATE,
            'prompt': TASK_TEMPLATE.format(text=row.content)
        }).json()['response'].strip()
            
    except json.JSONDecodeError:
        print("Invalid JSON response, skipping to next sample.")
    

0it [00:00, ?it/s]

In [45]:
predictions

{('661d1639b8beabb58229451b', '66252255616a4cb7061ee32c'): '1'}

In [14]:
predicted: pd.DataFrame = user_content.join(pd.Series(predictions, name=TASK_NAME))
# predicted.to_csv(f'{CFG.report_dir}/prediction.{TASK_NAME}.csv')
predicted.head()

ValueError: cannot join with no overlapping index names